# Test Notebook

In diesem Notebook werden alle Sachen geschrieben, die: 
- ausprobiert werden
- gerade nicht mehr eingesetzt werden
- Nur einmal für Datenverarbeitung benutzt werden, die nicht sein müsste

### Server DB richtig formatieren


In [ ]:
import sqlite3
import pandas as pd
from db_connect import db_get_df, db_save_df

con = sqlite3.connect("/Volumes/Samsung_T5/Backup/transcripts_all.sqlite")
df = pd.read_sql_query(f"SELECT * FROM transcript_word_level_2237", con)
con.close()

In [ ]:
len(df)

In [ ]:
print(df.dtypes)
print(df["filename"].drop_duplicates(ignore_index=True))

In [ ]:
df['filename'] = df['filename'].str.replace('/nfs/scratch/students/neumannvi84434/Podcast_Episoden/', '')

In [ ]:
with sqlite3.connect("/Volumes/Samsung_T5/Backup/transcripts.sqlite") as con:
    df.to_sql("transcript_word_level_2237", con, index=False, if_exists='replace')

In [ ]:
db_save_df(df, "transcript_word_level_2237")

### Satzbildung durch Whisper Punkte

Wenn wir nun alle MP3 Datein transkribiert haben, ist für jedes einzelne vorkommende Wort ein Zeitstempel gespeichert.

Für das Projekt lohnt es sich allerdings größere Abschnitte zu erstellen, auf die später die Embeddings und die Suchen angewendet werden können, um mehr Kontext miteinzubeziehen.

In [ ]:
from db_connect import db_get_df, db_save_df
from tqdm import tqdm
import pandas as pd

In [ ]:
df = db_get_df("transcript_word_level_2237")

In [ ]:
one_sentence = []
sentence_dict = [] 
filenames = df["filename"].drop_duplicates(ignore_index=True)

for filename in tqdm(filenames):
    word_entries = df[df['filename'] == filename]
    start = -1
    end = -1
    for index, word_entry in word_entries.iterrows(): 
        word = word_entry["word"]
        
        if not one_sentence: ##start
            start = word_entry["start"]
        one_sentence.append(word.strip())
        if "." in word:
            end = word_entry["end"]
            sentence = " ".join(one_sentence)
            sentence_dict.append({"filename": filename, "sentence": sentence, "start": start, "end":end})
            one_sentence = []



In [ ]:
df_sentences = pd.DataFrame(sentence_dict)

In [ ]:
len(df_sentences)

In [ ]:
db_save_df(df_sentences, "transcript_sentences_whisper")

### Setences ID hinzufügen

In [ ]:
df =db_get_df("sentences_compound_split")

In [ ]:
df['segment_id'] = df.groupby('filename').cumcount()

In [ ]:
db_save_df(df, "sentences_compound_split")

### Kompositatrennung mit Pyphen (Silbentrennung)

In [ ]:
import pyphen
import os

deutsche_worte = pyphen.Pyphen(lang='de_DE')
einzelwoerter_set = set()

dateipfad = os.path.join(DATA_PATH, "vocabulary.txt")

with open(dateipfad, 'r', encoding='utf-8') as datei:
    for zeile in datei:
        woerter = zeile.strip().split()
        
        for wort in woerter:
            getrenntes_wort = deutsche_worte.inserted(wort)
            
            einzelwoerter = getrenntes_wort.split('-')
            einzelwoerter_set.update(einzelwoerter)

for wort in einzelwoerter_set:
    print(wort)


In [ ]:
print(len(einzelwoerter_set))
print(einzelwoerter_set[3000:3010])

In [ ]:
def save_list_to_file(lst, file_path):
    with open(file_path, 'w') as file:
        for item in lst:
            file.write(str(item) + '\n')

In [ ]:
einzelwoerter_set = sorted(einzelwoerter_set)
save_list_to_file(einzelwoerter_set, os.path.join(DATA_PATH, "vocabulary_pyphen.txt"))

### Vokabular compound splitten

In [ ]:
from german_compound_splitter import comp_split
from embedding_creation.embedding_creator_TF_IDF import is_number, compound_split_sentence
from tqdm import tqdm
import os
from dotenv import load_dotenv

load_dotenv()
AUDIO_SOURCE_PATH = os.getenv("AUDIO_SOURCE_PATH")
DATA_PATH = os.getenv("DATA_PATH")

In [ ]:
einzelwoerter_set = set()
dateipfad = os.path.join(DATA_PATH, "vocabulary.txt")
input_file = os.path.join(DATA_PATH, "german.dic")
ahocs = comp_split.read_dictionary_from_file(input_file)

In [ ]:
with open(dateipfad, 'r', encoding='utf-8') as datei:

    for zeile in tqdm(datei):
        word = zeile.strip()
        if is_number(word):
            continue
        try:
            dissection = comp_split.dissect(word, ahocs, make_singular=True)
        except:
            dissection = ""
        for split_word in dissection:
            einzelwoerter_set.add(split_word)

print(len(einzelwoerter_set))
# print(einzelwoerter_set)

In [ ]:
einzelwoerter_set = sorted(einzelwoerter_set)
save_list_to_file(einzelwoerter_set, os.path.join(DATA_PATH, "vocabulary_compound_split.txt"))